# Filter Tiles

In [ ]:
import geopandas as gpd
import rioxarray
import pandas as pd
from rasterio import features
from shapely.geometry import shape
import folium
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load Data

gebco = rioxarray.open_rasterio('/home/jovyan/data/public/bathymetry/gebco/gebco_2025/gebco_2025.tif')
land = gpd.read_file('/home/jovyan/data/public/boundaries/indonesia_dissolved_simplified.geojson')
land_water_eez = gpd.read_file('/home/jovyan/data/public/boundaries/indonesia_simplified_land_water_aoi.geojson')
grid = gpd.read_file('s2_l2a_2024--P1Y-2024--P1Y.geojson') 

In [ ]:
# Transform CRS
land = land.to_crs(6933)
land_water_eez.to_crs(6933)
grid = grid.to_crs(6933)

In [ ]:
# Display Land and Grid
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Indonesia Land Boundaries
ax0 = axes[0]
land.plot(ax=ax0, facecolor='lightgreen', edgecolor='darkgreen', linewidth=0.5)
ax0.set_title('Indonesia Land Boundaries')
ax0.grid(True, alpha=0.3)

# Plot 2: Sentinel-2 L2A Grid
ax1 = axes[1]
grid.plot(ax=ax1, facecolor='none', edgecolor='red', linewidth=0.5)
ax1.set_title('Sentinel-2 L2A Grid (2024)')
ax1.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Clip GEBCO to Land-Water-EEZ boundary
gebco_clipped = gebco.rio.clip(land_water_eez.geometry.values, land_water_eez.crs, drop=True)

In [ ]:
# Display clipped GEBCO
gebco_clipped.plot(cmap='YlGnBu_r', vmin=-500, vmax=2, figsize=(6,3))
plt.show()

In [ ]:
# Extract shallow water areas + atolls from GEBCO

shallow_threshold = -100 
shallow_mask = (gebco_clipped.values[0] >= shallow_threshold) & (gebco_clipped.values[0] < 2)


In [ ]:
# Convert to polygon
transform = gebco_clipped.rio.transform()
shapes_gen = features.shapes(
    shallow_mask.astype(np.int16),
    mask=shallow_mask,
    transform=transform
)
shallow_geoms = [shape(geom) for geom, val in shapes_gen if val == 1]

In [ ]:
# Combine land and shallow water areas
combined = list(land.geometry) + shallow_geoms
aoi = gpd.GeoDataFrame({'geometry': combined}, crs=6933)

In [ ]:
selected = grid[grid.intersects(aoi.unary_union)]

In [ ]:
selected.plot()